# HybridMedNet - 医学影像深度学习诊断框架

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alltobebetter/HybridMedNet/blob/main/HybridMedNet_Colab.ipynb)

这个 notebook 可以让你在 Google Colab 上快速开始使用 HybridMedNet 进行医学影像诊断。

## 1. 环境设置

首先检查 GPU 是否可用，并克隆项目代码。

In [ ]:
# 检查 GPU
!nvidia-smi

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# 克隆项目
import os
if not os.path.exists('HybridMedNet'):
    !git clone https://github.com/alltobebetter/HybridMedNet.git
    print("[OK] 项目克隆完成")
else:
    print("[OK] 项目已存在")

%cd HybridMedNet

## 2. 安装依赖

安装所需的 Python 包。

In [ ]:
# 安装核心依赖
!pip install -q torch torchvision torchaudio
!pip install -q timm albumentations grad-cam einops
!pip install -q pandas matplotlib seaborn scikit-learn tqdm tensorboard
!pip install -q opencv-python pillow

# 可选：医学影像处理库
!pip install -q nibabel pydicom SimpleITK

print("[OK] 依赖安装完成")

## 3. 验证安装

确保所有模块都能正常导入。

In [ ]:
# 验证导入
import sys
import os

# 如果需要，添加项目路径
# sys.path.append('/content/HybridMedNet')

try:
    from models.hybrid_med_net import HybridMedNet
    from configs.default_config import Config
    from utils.metrics import calculate_metrics
    from utils.visualization import plot_training_curves
    from data.chest_xray_dataset import ChestXrayDataset, create_sample_dataset
    from data.transforms import get_train_transforms, get_val_transforms
    print("[OK] 所有模块导入成功")
except ImportError as e:
    print(f"[ERROR] 导入错误: {e}")
    print("请确保项目文件已正确上传到 Colab")

## 4. 快速开始 - 使用示例数据

### 4.0 设置项目路径

In [ ]:
# 确保 Python 能找到项目模块
import sys
import os

# 获取当前工作目录
current_dir = os.getcwd()
print(f"当前目录: {current_dir}")

# 添加项目根目录到 Python 路径
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)
    print(f"[OK] 已添加项目路径")

# 验证目录结构
print("\n项目结构:")
!ls -l

### 4.1 创建示例数据集

In [ ]:
# 创建示例数据集
from data.chest_xray_dataset import create_sample_dataset

print("正在创建示例数据集...")
data_dir = './data/sample_chest_xray'
labels_file = create_sample_dataset(data_dir, num_samples=200)
print(f"[OK] 示例数据集已创建: {data_dir}")

### 4.2 配置模型

In [ ]:
# 配置模型（针对 Colab 优化）
from configs.default_config import Config

config = Config()

# Colab 优化配置
config.TRAIN['batch_size'] = 16  # 减小 batch size 避免 OOM
config.TRAIN['epochs'] = 10      # 快速演示用较少 epoch
config.TRAIN['mixed_precision'] = True  # 使用混合精度训练
config.DATA['num_workers'] = 2   # Colab 上减少 workers

# 可以选择不同的 backbone
# config.MODEL['backbone'] = 'resnet50'  # 经典架构，更快
config.MODEL['backbone'] = 'convnext_base'  # 现代架构，更准确

print("配置信息：")
print(f"  Backbone: {config.MODEL['backbone']}")
print(f"  Batch size: {config.TRAIN['batch_size']}")
print(f"  Epochs: {config.TRAIN['epochs']}")
print(f"  Learning rate: {config.TRAIN['learning_rate']}")

### 4.3 训练模型（一键运行版本）

In [ ]:
# 使用 train.py 脚本训练
print("开始训练...\n")

# 直接运行 train.py
!python train.py

print("\n[OK] 训练完成")
print("模型已保存到: checkpoints/best_model.pth")

### 4.3.1 或者在 Notebook 中训练（高级用户）

In [ ]:
# 如果你想在 notebook 中直接训练，需要先确保 data 模块存在
# 由于 data 模块文件不在 GitHub 中，建议使用上面的 train.py 方式

# 以下代码仅供参考（需要 data 模块）
'''
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from models.hybrid_med_net import HybridMedNet
from data.chest_xray_dataset import ChestXrayDataset
from data.transforms import get_train_transforms, get_val_transforms

# 创建数据集
train_dataset = ChestXrayDataset(
    data_dir=config.DATA['data_dir'],
    labels_file=config.DATA['labels_file'],
    transform=get_train_transforms(config.DATA['image_size']),
    split='train'
)

val_dataset = ChestXrayDataset(
    data_dir=config.DATA['data_dir'],
    labels_file=config.DATA['labels_file'],
    transform=get_val_transforms(config.DATA['image_size']),
    split='val'
)

train_loader = DataLoader(
    train_dataset,
    batch_size=config.TRAIN['batch_size'],
    shuffle=True,
    num_workers=config.DATA['num_workers']
)

val_loader = DataLoader(
    val_dataset,
    batch_size=config.TRAIN['batch_size'],
    shuffle=False,
    num_workers=config.DATA['num_workers']
)

# ... 训练代码 ...
'''

print("[INFO] 建议使用上面的 train.py 方式进行训练")

### 4.4 可视化训练过程

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.show()

## 5. 评估模型

In [ ]:
# 评估模型
!python evaluate.py --model_path best_model.pth --save_results

或者在 notebook 中评估：

In [ ]:
from utils.metrics import calculate_metrics
from sklearn.metrics import roc_auc_score, roc_curve
import numpy as np

# 加载最佳模型
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

# 收集预测结果
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in tqdm(val_loader, desc="Evaluating"):
        images = images.to(device)
        outputs = model(images)
        preds = torch.sigmoid(outputs).cpu().numpy()
        
        all_preds.append(preds)
        all_labels.append(labels.numpy())

all_preds = np.vstack(all_preds)
all_labels = np.vstack(all_labels)

# 计算指标（转换为二值预测）
all_preds_binary = (all_preds > 0.5).astype(int)
metrics = calculate_metrics(all_labels, all_preds_binary, all_preds)

print("\n评估结果:")
print(f"准确率: {metrics['accuracy']:.4f}")
print(f"精确率: {metrics['precision']:.4f}")
print(f"召回率: {metrics['recall']:.4f}")
print(f"F1分数: {metrics['f1']:.4f}")
print(f"平均AUC: {metrics['auc']:.4f}")

## 6. 预测示例图像（自动运行）

In [ ]:
# 使用验证集中的图像进行预测演示
from PIL import Image
import os

# 获取几张示例图像
sample_images = [os.path.join(config.DATA['data_dir'], f) 
                 for f in os.listdir(config.DATA['data_dir']) 
                 if f.endswith(('.jpg', '.png'))][:3]

class_names = train_dataset.class_names

for img_path in sample_images:
    image = Image.open(img_path).convert('RGB')
    
    # 预处理
    transform = get_val_transforms(config.DATA['image_size'])
    image_tensor = transform(image).unsqueeze(0).to(device)
    
    # 预测
    model.eval()
    with torch.no_grad():
        output = model(image_tensor)
        probs = torch.sigmoid(output).cpu().numpy()[0]
    
    # 显示结果
    plt.figure(figsize=(12, 4))
    
    # 显示图像
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title(f'Input: {os.path.basename(img_path)}')
    plt.axis('off')
    
    # 显示预测概率
    plt.subplot(1, 2, 2)
    y_pos = np.arange(len(class_names))
    plt.barh(y_pos, probs)
    plt.yticks(y_pos, class_names)
    plt.xlabel('Probability')
    plt.title('Predictions')
    plt.xlim([0, 1])
    
    plt.tight_layout()
    plt.show()
    
    # 打印 Top-3 预测
    top3_idx = np.argsort(probs)[-3:][::-1]
    print(f"\nTop-3 预测:")
    for idx in top3_idx:
        print(f"{class_names[idx]}: {probs[idx]:.2%}")
    print("\n" + "="*50 + "\n")

## 7. 使用真实数据集（可选 - 一键下载）

### 选项 1: 胸部 X 光肺炎数据集（推荐，小而快）

**数据集**: Chest X-Ray Images (Pneumonia)  
**大小**: ~1.2 GB  
**图像数**: 5,863 张  
**类别**: Normal / Pneumonia  
**来源**: Kaggle

In [ ]:
# 安装下载工具
!pip install -q gdown

# 下载数据集
print("正在下载胸部 X 光肺炎数据集...")
!mkdir -p ./data/chest_xray

# 使用公开链接下载
!wget -q --show-progress https://data.mendeley.com/public-files/datasets/rscbjbr9sj/files/f12eaf6d-6023-432f-acc9-80c9d7393433/file_downloaded -O chest_xray.zip

# 解压
print("\n正在解压...")
!unzip -q chest_xray.zip -d ./data/chest_xray
!rm chest_xray.zip

print("\n[OK] 数据集下载完成")
print(f"数据位置: ./data/chest_xray/")

# 查看数据结构
!ls -lh ./data/chest_xray/

### 选项 2: COVID-19 胸部 X 光数据集（中等大小）

**数据集**: COVID-19 Radiography Database  
**大小**: ~1.5 GB  
**图像数**: 21,165 张  
**类别**: COVID / Normal / Viral Pneumonia / Lung Opacity  
**来源**: Kaggle

In [ ]:
# 下载 COVID-19 数据集
print("正在下载 COVID-19 胸部 X 光数据集...")
!mkdir -p ./data/covid19

# 使用 gdown 从 Google Drive 下载
!gdown --id 1-HQQciKYfwAO3oH7ci5sD5NBPbxXXGs4 -O covid19.zip

# 解压
print("\n正在解压...")
!unzip -q covid19.zip -d ./data/covid19
!rm covid19.zip

print("\n[OK] COVID-19 数据集下载完成")
print(f"数据位置: ./data/covid19/")

# 查看数据结构
!ls -lh ./data/covid19/

### 选项 3: 皮肤病变数据集 ISIC（小而快）

**数据集**: ISIC 2019 Skin Lesion  
**大小**: ~500 MB（采样版本）  
**图像数**: 2,000+ 张  
**类别**: 8 种皮肤病变类型  
**来源**: ISIC Archive

In [ ]:
# 下载 ISIC 皮肤病变数据集（采样版本）
print("正在下载 ISIC 皮肤病变数据集...")
!mkdir -p ./data/isic

# 从公开源下载
!wget -q --show-progress https://isic-challenge-data.s3.amazonaws.com/2019/ISIC_2019_Training_Input.zip -O isic_images.zip
!wget -q --show-progress https://isic-challenge-data.s3.amazonaws.com/2019/ISIC_2019_Training_GroundTruth.csv -O ./data/isic/labels.csv

# 解压
print("\n正在解压...")
!unzip -q isic_images.zip -d ./data/isic/images
!rm isic_images.zip

print("\n[OK] ISIC 数据集下载完成")
print(f"数据位置: ./data/isic/")

# 查看数据
!ls -lh ./data/isic/
!head -5 ./data/isic/labels.csv

### 选项 4: 脑肿瘤 MRI 数据集（最小）

**数据集**: Brain Tumor MRI Dataset  
**大小**: ~150 MB  
**图像数**: 3,000+ 张  
**类别**: Glioma / Meningioma / Pituitary / No Tumor  
**来源**: Kaggle

In [ ]:
# 下载脑肿瘤 MRI 数据集
print("正在下载脑肿瘤 MRI 数据集...")
!mkdir -p ./data/brain_tumor

# 使用 gdown 下载
!gdown --id 1A2IU8Sgea1h3fYLpYtFb2v7NYdMjvEhP -O brain_tumor.zip

# 解压
print("\n正在解压...")
!unzip -q brain_tumor.zip -d ./data/brain_tumor
!rm brain_tumor.zip

print("\n[OK] 脑肿瘤数据集下载完成")
print(f"数据位置: ./data/brain_tumor/")

# 查看数据结构
!ls -lh ./data/brain_tumor/
!find ./data/brain_tumor/ -type d

### 使用下载的真实数据集训练

根据你选择的数据集，更新配置并重新训练：

In [ ]:
# 选择你下载的数据集
DATASET_CHOICE = 'chest_xray'  # 可选: 'chest_xray', 'covid19', 'isic', 'brain_tumor'

if DATASET_CHOICE == 'chest_xray':
    config.DATA['data_dir'] = './data/chest_xray/chest_xray/train'
    config.MODEL['num_classes'] = 2  # Normal / Pneumonia
    print("[OK] 使用胸部 X 光肺炎数据集")
    
elif DATASET_CHOICE == 'covid19':
    config.DATA['data_dir'] = './data/covid19/COVID-19_Radiography_Dataset'
    config.MODEL['num_classes'] = 4  # COVID / Normal / Viral / Opacity
    print("[OK] 使用 COVID-19 数据集")
    
elif DATASET_CHOICE == 'isic':
    config.DATA['data_dir'] = './data/isic/images'
    config.DATA['labels_file'] = './data/isic/labels.csv'
    config.MODEL['num_classes'] = 8  # 8 种皮肤病变
    print("[OK] 使用 ISIC 皮肤病变数据集")
    
elif DATASET_CHOICE == 'brain_tumor':
    config.DATA['data_dir'] = './data/brain_tumor/Training'
    config.MODEL['num_classes'] = 4  # 4 种脑肿瘤类型
    print("[OK] 使用脑肿瘤 MRI 数据集")

# 调整训练参数
config.TRAIN['epochs'] = 30  # 真实数据需要更多 epoch
config.TRAIN['batch_size'] = 32
config.MODEL['backbone'] = 'resnet50'  # 使用更快的 backbone

print(f"\n配置更新:")
print(f"数据目录: {config.DATA['data_dir']}")
print(f"类别数: {config.MODEL['num_classes']}")
print(f"训练轮数: {config.TRAIN['epochs']}")
print(f"Backbone: {config.MODEL['backbone']}")

# 现在可以重新运行训练 cell（4.3）来使用真实数据训练

## 8. 实验不同的配置

### 8.1 尝试不同的 Backbone

In [ ]:
# 经典架构（更快）
backbones_classic = ['resnet34', 'resnet50', 'resnet101']

# 现代架构（更准确）
backbones_modern = [
    'convnext_tiny', 'convnext_small', 'convnext_base',
    'swin_tiny', 'swin_small',
    'efficientnetv2_s', 'efficientnetv2_m'
]

# 选择一个 backbone 进行实验
config.MODEL['backbone'] = 'convnext_small'
print(f"使用 backbone: {config.MODEL['backbone']}")

### 8.2 调整训练参数

In [ ]:
# 实验不同的学习率
learning_rates = [1e-5, 3e-5, 1e-4, 3e-4]

# 实验不同的 batch size
batch_sizes = [8, 16, 32, 64]

# 选择配置
config.TRAIN['learning_rate'] = 1e-4
config.TRAIN['batch_size'] = 16

print(f"学习率: {config.TRAIN['learning_rate']}")
print(f"Batch size: {config.TRAIN['batch_size']}")

## 9. 故障排除

### GPU 内存不足

In [ ]:
# 清理 GPU 内存
import gc
torch.cuda.empty_cache()
gc.collect()

# 减小 batch size
config.TRAIN['batch_size'] = 8

# 使用梯度累积
config.TRAIN['gradient_accumulation_steps'] = 4

print("[OK] GPU 内存已清理")

## 总结

这个 notebook 展示了如何在 Google Colab 上使用 HybridMedNet：

1. ✅ 环境设置和依赖安装
2. ✅ 使用示例数据快速训练
3. ✅ 模型评估和可视化
4. ✅ 预测新图像
5. ✅ 使用真实医学数据集（可选）

### 下一步

- 尝试不同的 backbone 架构
- 调整超参数优化性能
- 使用真实医学数据���训练
- 部署模型为 Web 服务

### 资源链接

- [项目 GitHub](https://github.com/alltobebetter/HybridMedNet)
- [完整文档](https://github.com/alltobebetter/HybridMedNet/blob/main/README.md)
- [快速开始指南](https://github.com/alltobebetter/HybridMedNet/blob/main/QUICKSTART.md)